In [5]:
import numpy as np
from scipy import sparse, stats
import pandas as pd
import bottleneck
import gc
import warnings
import scanpy as sc
import anndata as ad
import time
import os
from scipy import stats, sparse
import concurrent.futures
import statistics
import random
import threading
import concurrent.futures
from scipy.sparse import csr_matrix
from scipy.stats import zscore

from pyprojroot import here
import pymn
import resource

In [6]:
here()
base_data_folder = "/vault/lfrench/mouse_brain_cluster_replicability/data/"

In [7]:
#common universe of markers
cluster_annotations = pd.read_csv(base_data_folder + "/whole_mouse_brain/zeng/from_aws/AIT21.0/AIT21_annotation_freeze_081523.tsv", sep="\t")
all_markers = set(cluster_annotations['cluster.markers'].str.split(',').explode().tolist())
all_Zeng_markers = [x for x in all_markers if pd.notna(x)]
all_Zeng_markers = set(all_Zeng_markers)
#load Macosko universe and markers



In [8]:
#load Macosko marker lists from preprint supplement table
Macosko_supplement_table = pd.read_excel(base_data_folder + "/whole_mouse_brain/processed/macosko/paper_supplements/41586_2023_6818_MOESM10_ESM.xlsx", sheet_name = "Whole Brain Set Cover")
#just the most specific markers
Macosko_supplement_table = Macosko_supplement_table[Macosko_supplement_table["Exclude up to N Nearest Neighbors"] == 0]

Macosko_supplement_table = Macosko_supplement_table[["Cell Type Name", "Optimally-Sized Gene List"]]
Macosko_markers = Macosko_supplement_table['Optimally-Sized Gene List'].str.split('|', expand=True)
Macosko_markers = pd.concat([Macosko_supplement_table['Cell Type Name'], Macosko_markers], axis=1)
Macosko_markers = pd.melt(Macosko_markers, id_vars = 'Cell Type Name', value_name='gene_symbol', var_name='order')
Macosko_markers['gene_symbol'] = Macosko_markers['gene_symbol'].str.replace("=.*", "", regex=True)
Macosko_markers.dropna(subset=['gene_symbol'], inplace=True)
Macosko_markers = Macosko_markers.rename(columns = {"gene_symbol":"gene"})
Macosko_markers = Macosko_markers.rename(columns = {"Cell Type Name":"Macosko_cluster"})


In [9]:
all_Macosko_markers = set(Macosko_markers.gene)

In [10]:
len(all_Macosko_markers)

2706

In [11]:
len(all_Zeng_markers)

2725

In [ ]:
#for subsetting
#all_Zeng_cpm = sc.read_h5ad(base_data_folder + "/whole_mouse_brain/processed/zeng/subsets/AIT21.0.merged.with_multiome_all_genes.cpm.h5ad") 



In [13]:
#for when it's already been subsetted
#all_Zeng_cpm = sc.read_h5ad(base_data_folder + "/whole_mouse_brain/processed/zeng/subsets/AIT21.0.merged.with_multiome_3309_markers_only.cpm.h5ad") 
all_Zeng_cpm = sc.read_h5ad(base_data_folder + "/whole_mouse_brain/processed/zeng/subsets/AIT21.0.merged.with_multiome_3820_markers_only.cpm.h5ad") 

In [14]:
adata_macosko = sc.read_h5ad(base_data_folder + "/whole_mouse_brain/macosko/from_google_drive/Macosko_Mouse_Atlas_Single_Nuclei.Use_Backed.h5ad", backed = "r")
gene_map = adata_macosko.var
Macosko_universe = set(gene_map['gene_name'].to_list())
Zeng_universe = set(all_Zeng_cpm.var.gene_symbol.to_list())

In [15]:
#join with universes
all_Macosko_markers = all_Macosko_markers.intersection(Zeng_universe)

In [16]:
len(all_Macosko_markers)

2673

In [17]:
len(all_Zeng_markers)

2725

In [18]:
len(all_Macosko_markers.intersection(all_Zeng_markers))

1578

In [19]:
len(Macosko_universe.intersection(Zeng_universe))

3309

In [20]:
combined_markers = all_Zeng_markers.union(all_Macosko_markers)

In [21]:
len(combined_markers)

3820

In [ ]:
##subset to marker genes, and write out, used once
#all_Zeng_cpm.var["is_Marker"] = all_Zeng_cpm.var.gene_symbol.isin(combined_markers)
#all_Zeng_cpm = all_Zeng_cpm[:, all_Zeng_cpm.var["is_Marker"]]
#all_Zeng_cpm.write_h5ad(base_data_folder + "/whole_mouse_brain/processed/zeng/subsets/AIT21.0.merged.with_multiome_3820_markers_only.cpm.h5ad") 

In [22]:
if not os.path.isdir(os.path.join(here(), "results", "marker_results")):
  os.mkdir(os.path.join(here(), "results", "marker_results"))

In [23]:
Zeng_cluster_info_AWS = pd.read_csv(base_data_folder + "/whole_mouse_brain/zeng/from_aws/AIT21.0/AIT21_annotation_freeze_081523.tsv", sep = '\t')
#remove low quality cells
LQ_clusters = Zeng_cluster_info_AWS.cl[Zeng_cluster_info_AWS['subclass_label'] == "LQ"].to_list()
LQ_clusters = [str(i) for i in LQ_clusters]
all_Zeng_cpm.obs["is_LQ"] = all_Zeng_cpm.obs.cl.isin(LQ_clusters)
all_Zeng_cpm = all_Zeng_cpm[~all_Zeng_cpm.obs["is_LQ"], :]


In [24]:
all_Zeng_cpm

View of AnnData object with n_obs × n_vars = 4042976 × 3819
    obs: 'library_prep', 'gene.counts.0', 'doublet_score', 'roi', 'umi.counts', 'method', 'sex', 'external_donor_name', 'age', 'medical_conditions', 'cl', 'is_LQ'
    var: 'gene_symbol', 'gene_identifier', 'is_Marker'

In [22]:
#%%time
all_Zeng_cpm_X = sparse.csr_matrix(all_Zeng_cpm.X)
df = pd.DataFrame.sparse.from_spmatrix(all_Zeng_cpm_X)
df.index = all_Zeng_cpm.obs.index  # Assuming row labels are stored in `.obs_names`
df.columns = all_Zeng_cpm.var.index

In [23]:
#%%time
numeric_cols = df.select_dtypes(include=[np.number]).columns
for col in numeric_cols:
    df[col] = zscore(df[col].to_numpy())

In [24]:
all_clusters_Zeng = Zeng_cluster_info_AWS.cl.tolist()
all_clusters_Zeng = [str(i) for i in all_clusters_Zeng]

In [25]:
#restrict to what we have cells for - handles LQ clusters
all_clusters_Zeng = list(set(all_clusters_Zeng).intersection(all_Zeng_cpm.obs.cl))

In [26]:
len(set(all_Zeng_cpm.obs.cl))

5322

In [27]:
len(all_clusters_Zeng)

5322

In [28]:
all_clusters_Macosko = Macosko_markers.Macosko_cluster.tolist()

In [29]:
#for testing
#all_clusters_Macosko = all_clusters_Macosko[1:100]
#all_clusters_Zeng = all_clusters_Zeng[1:100]

In [30]:
all_clusters= list(set(all_clusters_Macosko + all_clusters_Zeng)) #make unique

In [31]:
#make dict for cluster markers and cells in cluster
cluster_marker_dict = dict()
for cluster in all_clusters_Zeng:
    #get markers 
    cluster_markers = Zeng_cluster_info_AWS[Zeng_cluster_info_AWS.cl == int(cluster)][['cluster.markers']].iloc[0,0]
    cells_in_cluster = all_Zeng_cpm.obs[all_Zeng_cpm.obs.cl == cluster].index.tolist()
    if pd.isna(cluster_markers) or len(cells_in_cluster) == 0:
        continue
    cluster_markers = cluster_markers.split(',')
    cluster_markers = list(set(cluster_markers).intersection(all_Zeng_cpm.var.index.to_list()))
    cluster_marker_dict[cluster] = cluster_markers


In [32]:
#Add in Macosko markers too
for cluster in all_clusters_Macosko:
    #get markers 
    cluster_markers = Macosko_markers[Macosko_markers.Macosko_cluster == cluster].gene.tolist()
    cluster_markers = list(set(cluster_markers).intersection(combined_markers))
    cluster_marker_dict[cluster] = cluster_markers

In [33]:
len(cluster_marker_dict)

455

In [34]:
cluster_marker_dict

{'1744': ['Tll2', 'Kcnmb1'],
 '2532': ['Col6a3', 'Cck', 'Zeb2', '6430628N08Rik', 'Emx2'],
 '4308': ['Col14a1', 'Prrxl1', 'Cpa6', 'Phox2b'],
 '4924': ['Dkk1', 'Gm29771'],
 '1685': ['Bdkrb2', 'Lhx1', 'Csta2'],
 '5212': ['Pdzd2', 'Plekhg1', 'Gpr17', 'Gp1bb'],
 '2361': ['St18', 'Pi15', 'Pitx2', 'Rbm20'],
 '2678': ['Otp', 'Gm15482'],
 '2044': ['Gm35721', 'Trh', 'Tbx19'],
 '1761': ['Opn5', 'Gm13264'],
 '3222': ['B020031H02Rik', 'Calca', 'Glp1r', 'Kcnmb2'],
 '3822': ['Hmcn1', 'D130079A08Rik', 'Evx1os'],
 '24': ['Drd5', '0610040J01Rik', 'Col24a1', 'Gm50370'],
 '99': ['Col15a1', 'Rxfp1', 'Inhba', 'Gm49678', 'Egr3', 'Colq'],
 '2421': ['Crhbp', 'Slc26a7', 'Abcc9'],
 '20': ['Myzap', 'Gm19744', 'Lefty2', 'Prph'],
 '3968': ['Npff', 'Olfr574'],
 '3088': ['Nkx6-1', 'Gm40663'],
 '4970': ['Pitx1', 'Ebf3', 'Tfap2d'],
 '94': ['2310002F09Rik', 'Fn1', 'Cobll1', 'Pirt'],
 '2066': ['Gm30373', 'Gsx1', 'Vwc2l'],
 '4162': ['Pthlh', 'Aldh1a3', 'Pvalb'],
 '3903': ['Arhgap36', 'Npnt', 'Pamr1', 'Phox2b'],
 '438': ['

In [35]:
# Flatten the dictionary and convert it to a DataFrame
cluster_marker_dict_long = pd.DataFrame([(k, v) for k, vs in cluster_marker_dict.items() for v in vs], columns=['cluster', 'gene_symbol'])

# Create a new DataFrame with 1s where the values match
cluster_marker_dict_table = cluster_marker_dict_long.pivot_table(index='cluster', columns='gene_symbol', aggfunc=len, fill_value=0)



In [36]:
cluster_marker_dict_table.shape

(455, 931)

In [37]:
df[cluster_marker_dict_table.columns].shape

(4042976, 931)

In [38]:
#line up the df dataframe rows
cluster_sums = df[cluster_marker_dict_table.columns] @ cluster_marker_dict_table.T

In [39]:
cluster_sums.shape

(4042976, 455)

In [40]:
#%%time
one_hot_cluster_membership = pd.get_dummies(all_Zeng_cpm.obs.cl)

In [41]:
#restrict to cluster sum clusters

In [42]:
one_hot_cluster_membership = one_hot_cluster_membership[all_clusters_Zeng]

In [43]:
#AUROC code based on https://github.com/gillislab/pyMN/blob/master/pymn/utils.py

In [44]:
#%%time
ranks = bottleneck.rankdata(cluster_sums, axis = 0)

In [45]:
sum_pos_ranks = one_hot_cluster_membership.T @ ranks

In [46]:
n_pos = bottleneck.nansum(one_hot_cluster_membership, axis=0)

In [47]:
n_neg = one_hot_cluster_membership.shape[0] - n_pos

In [48]:
roc = sum_pos_ranks / n_pos[:, None]

In [49]:
roc -= (n_pos[:, None] + 1) / 2

In [50]:
roc /= n_neg[:, None]

In [51]:
roc.head()

,0,1,2,3,4,5,6,7,8,9,...,445,446,447,448,449,450,451,452,453,454
1744,0.496571,0.456734,0.364037,0.506770,0.375281,0.495348,0.464857,0.458315,0.834441,0.453283,...,0.768735,0.454685,0.424782,0.762748,0.439853,0.900314,0.489657,0.485403,0.457211,0.792253
2532,0.616124,0.476778,0.463821,0.871147,0.509941,0.637233,0.536173,0.734588,0.810642,0.440469,...,0.491161,0.449476,0.440518,0.578184,0.467038,0.886885,0.493967,0.485409,0.459988,0.621666
4308,0.550427,0.443503,0.377776,0.899281,0.576248,0.674385,0.652659,0.633390,0.716057,0.477872,...,0.638047,0.461060,0.417645,0.470555,0.461712,0.748658,0.489657,0.482592,0.457212,0.452014
4924,0.481440,0.443503,0.643130,0.719631,0.687747,0.456234,0.529594,0.722890,0.596205,0.769583,...,0.812807,0.492749,0.439989,0.495044,0.439320,0.619155,0.499759,0.482592,0.457212,0.537402
1685,0.496949,0.453143,0.446361,0.809482,0.369686,0.677487,0.585264,0.577569,0.751371,0.448294,...,0.732941,0.438651,0.433388,0.437562,0.523620,0.469174,0.489657,0.482592,0.462063,0.383429


In [52]:
#set row and column labels

In [53]:
roc.index = roc.index.map(lambda x: x + '_cluster')
roc.columns = cluster_sums.columns.tolist()
roc.columns = roc.columns.map(lambda x: x + '_markers')

In [54]:
roc.head()

,1006_markers,1017_markers,1031_markers,1043_markers,1057_markers,1111_markers,1117_markers,1125_markers,1159_markers,1187_markers,...,Inh_Sox8_Calcrl_9_markers,Inh_Sp8_Ctsc_1_markers,Inh_Sp8_Ctsc_3_markers,Inh_Sp8_Vip_1_markers,NG_Myoz1_1_markers,NG_Myoz1_8_markers,NorEx_Phox2b_Ndufa4l2_1_markers,OPC_Cspg5_markers,SerInh_Fev_Fbxw28_markers,Ser_Fev_Samd9l_markers
1744_cluster,0.496571,0.456734,0.364037,0.506770,0.375281,0.495348,0.464857,0.458315,0.834441,0.453283,...,0.768735,0.454685,0.424782,0.762748,0.439853,0.900314,0.489657,0.485403,0.457211,0.792253
2532_cluster,0.616124,0.476778,0.463821,0.871147,0.509941,0.637233,0.536173,0.734588,0.810642,0.440469,...,0.491161,0.449476,0.440518,0.578184,0.467038,0.886885,0.493967,0.485409,0.459988,0.621666
4308_cluster,0.550427,0.443503,0.377776,0.899281,0.576248,0.674385,0.652659,0.633390,0.716057,0.477872,...,0.638047,0.461060,0.417645,0.470555,0.461712,0.748658,0.489657,0.482592,0.457212,0.452014
4924_cluster,0.481440,0.443503,0.643130,0.719631,0.687747,0.456234,0.529594,0.722890,0.596205,0.769583,...,0.812807,0.492749,0.439989,0.495044,0.439320,0.619155,0.499759,0.482592,0.457212,0.537402
1685_cluster,0.496949,0.453143,0.446361,0.809482,0.369686,0.677487,0.585264,0.577569,0.751371,0.448294,...,0.732941,0.438651,0.433388,0.437562,0.523620,0.469174,0.489657,0.482592,0.462063,0.383429


In [55]:
roc.shape

(229, 455)

In [56]:
timestamp = str(round(time.time()))

In [ ]:
roc.rename_axis('cluster_name').to_csv(os.path.join(here(), "results", "marker_results", "markers_on_Zeng." + timestamp + ".csv.gz"), compression="gzip")

In [57]:
print(timestamp)

1713300779


In [58]:
print("Done all markers!")

Done all markers!


In [59]:
#run each cluster against it's self and the top hit for it

In [60]:
#need to melt output matrix to get these two clusters, can probably run on dactyl

In [61]:
#needs to iterate all marker sets, get the two clusters, then compute AUROC 

In [62]:
#code duplication

In [84]:
roc_used_marker_lists = [string.replace("_markers", "") for string in roc.columns.tolist()]

In [90]:
#some are removed due to missing data - not sure why we drop these, maybe LQ
set(all_clusters_Zeng) - set(roc_used_marker_lists)

set()

In [91]:
all_clusters_Zeng = set(all_clusters_Zeng).intersection(set(roc_used_marker_lists))

In [92]:
len(all_clusters_Zeng)

226

In [93]:
all_results = []
for current_marker in all_clusters_Zeng:
    roc_series = roc[current_marker + "_markers"].copy()
    self_auroc = roc_series.loc[current_marker + "_cluster"] #only used for output
    #set to zero so it doesn't mask the non-self max
    roc_series.loc[current_marker + "_cluster"] = 0
    highest_non_self_cluster = roc_series.idxmax()
    two_cluster_membership = one_hot_cluster_membership[[highest_non_self_cluster.replace("_cluster", ""), current_marker]]
    two_cluster_membership = two_cluster_membership[two_cluster_membership.sum(axis=1) > 0]
    cluster_sums_self = cluster_sums.loc[two_cluster_membership.index, current_marker]
    ranks = bottleneck.rankdata(cluster_sums_self, axis = 0)
    sum_pos_ranks = two_cluster_membership[current_marker].T @ ranks
    n_pos = bottleneck.nansum(two_cluster_membership[current_marker], axis=0)
    if n_pos == 0:
        continue
    n_neg = two_cluster_membership[current_marker].shape[0] - n_pos
    single_roc = sum_pos_ranks / n_pos
    single_roc -= (n_pos + 1) / 2
    single_roc /= n_neg
    single_cluster_result = pd.DataFrame({'target_cluster' : current_marker,
                                      'highest_non_self_cluster' : highest_non_self_cluster,
                                      'max_AUROC' : roc_series.max(),
                                      'self_1_vs_all_AUROC' : self_auroc,
                                      'self_vs_best_AUROC' : single_roc}, index=[0]) 
    all_results.append(single_cluster_result)

all_results = pd.concat(all_results)

In [94]:
all_results

,target_cluster,highest_non_self_cluster,max_AUROC,self_1_vs_all_AUROC,self_vs_best_AUROC
0,4416,4207_cluster,0.994594,0.990745,0.325855
0,1744,4239_cluster,0.804730,0.921052,0.671334
0,686,5067_cluster,0.946456,0.973565,0.882398
0,1187,4185_cluster,0.959868,0.974710,0.645817
0,2532,5212_cluster,0.895114,0.951151,0.817293
...,...,...,...,...,...
0,2683,2871_cluster,0.990878,0.971497,0.405614
0,943,1187_cluster,0.961153,0.952613,0.560455
0,4741,2975_cluster,0.998973,0.995768,0.321462
0,3240,2863_cluster,0.999033,0.975437,0.186975


In [95]:
all_results.to_csv(os.path.join(here(), "results", "marker_results", "Zeng_self_vs_best." + timestamp + ".csv.gz"), compression="gzip", index=False)

In [99]:
print("Done!")

Done!
